In [24]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import subprocess
from IPython.display import Markdown, display

In [25]:
load_dotenv(override=True)

True

In [ ]:
openai_api_key = os.getenv("OPENAI_API_KEY")
claude_api_key  = os.getenv("CLAUDE_API_KEY")
google_api_key = os.getenv("GEMINI_API_KEY")

In [27]:
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if claude_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AI


In [28]:
openai = OpenAI()
claude = OpenAI(api_key=anthropic_api_key, base_url="https://api.anthropic.com/v1/")
gemini = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [30]:
OPENAI_MODEL = "gpt-5-mini"
CLAUDE_MODEL = "claude-sonnet-4-5-20250929"
GEMINI_MODEL = "gemini-2.5-flash-lite"

In [31]:
from system_info import retrieve_system_info

system_info = retrieve_system_info()
system_info

# this would print the system info of the user computer

{'os': {'system': 'Darwin',
  'arch': 'arm64',
  'release': '23.4.0',
  'version': 'Darwin Kernel Version 23.4.0: Fri Mar 15 00:19:22 PDT 2024; root:xnu-10063.101.17~1/RELEASE_ARM64_T8112',
  'kernel': '23.4.0',
  'distro': None,
  'wsl': False,
  'rosetta2_translated': False,
  'target_triple': 'arm64-apple-darwin23.4.0'},
 'package_managers': ['xcode-select (CLT)', 'brew'],
 'cpu': {'brand': 'Apple M2',
  'cores_logical': 8,
  'cores_physical': 8,
  'simd': []},
 'toolchain': {'compilers': {'gcc': 'Apple clang version 15.0.0 (clang-1500.3.9.4)',
   'g++': 'Apple clang version 15.0.0 (clang-1500.3.9.4)',
   'clang': 'Apple clang version 15.0.0 (clang-1500.3.9.4)',
   'msvc_cl': ''},
  'build_tools': {'cmake': '', 'ninja': '', 'make': 'GNU Make 3.81'},
  'linkers': {'ld_lld': ''}}}

In [32]:
system_prompt = """
Here is a report of the system information for my computer.
I want to run a C++ compiler to compile a single C++ file called main.cpp and then execute it in the simplest way possible.
Please reply with whether I need to install any C++ compiler to do this. If so, please provide the simplest step by step instructions to do so.

If I'm already set up to compile C++ code, then I'd like to run something like this in Python to compile and execute the code:
```python
compile_command = # something here - to achieve the fastest possible runtime performance
compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_command = # something here
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout
```
Please tell me exactly what I should use for the compile_command and run_command.

System information:
{system_info}
"""

In [33]:
response = openai.chat.completions.create(model=OPENAI_MODEL, messages=[{"role": "user", "content": message}])
display(Markdown(response.choices[0].message.content))

You already have a usable C++ compiler: the report shows Apple clang (clang / g++ wrappers) is installed on your M2 mac. So you do NOT need to install anything extra to compile a single file.

If for some reason clang isn't available at runtime, you can install the Command Line Tools with:
xcode-select --install
(or install a GCC toolchain via Homebrew with brew install gcc — not necessary here).

For fastest possible runtime from a single-file build while staying simple, use Apple clang with high optimization and LTO. In Python (using subprocess.run with a list, not a shell string) set:

compile_command = ["clang++",
                   "-std=c++20",
                   "-Ofast",
                   "-DNDEBUG",
                   "-flto",
                   "-o", "main",
                   "main.cpp"]

run_command = ["./main"]

Notes:
- "-Ofast" gives the most aggressive optimization (includes -O3 and fast-math tradeoffs).
- "-flto" enables link-time optimization which can give extra speed for single-binary builds.
- "-DNDEBUG" disables assert() checks.
- If a compile error occurs because the platform rejects any of these flags, drop "-flto" or "-Ofast" and try:
  ["clang++", "-std=c++20", "-O3", "-DNDEBUG", "-o", "main", "main.cpp"]

Example usage in your snippet (you already have a good pattern):

compile_result = subprocess.run(compile_command, check=True, text=True, capture_output=True)
run_result = subprocess.run(run_command, check=True, text=True, capture_output=True)
return run_result.stdout

That's all — with the system info you provided, the first compile_command above should work out of the box.

In [64]:
compile_command = ["clang++", "-std=c++20", "-O3", "-DNDEBUG", "-o", "main", "main.cpp"]

run_command = ["./main"]

In [65]:
system_prompt = """
Your task is to convert Python code into high performance C++ code.
Respond only with C++ code. Do not provide any explanation other than occasional comments.
The C++ response needs to produce an identical output in the fastest possible time.
"""

In [66]:
def user_prompt_for(python_code: str) -> str:
    return f"""
    Port this Python code to C++ with the fastest possible implementation that produces identical output in the least time.
    The system information is:
    {system_info}
    Your response will be written to a file called main.cpp and then compiled and executed; the compilation command is:
    {compile_command}
    
    Respond only with C++ code.
    Python code to port:

    ```python
    {python_code}
    ```
    """

In [67]:
def messages_for(python):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [68]:
def write_output(cpp):
    with open("main.cpp", "w", encoding="utf-8") as f:
        f.write(cpp)

In [69]:
def port(client, model, python):
    reasoning_effort = "high" if 'gpt' in model else None
    response = client.chat.completions.create(model=model, messages=messages_for(python), reasoning_effort=reasoning_effort)
    reply = response.choices[0].message.content
    reply = reply.replace('```cpp','').replace('```','')
    write_output(reply)

In [70]:
pi = """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(200_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
"""

In [71]:
def run_python(code):
    globals = {"__builtins__": __builtins__}
    exec(code, globals)

In [72]:
run_python(pi)

Result: 3.141592656089
Execution Time: 13.922018 seconds


In [73]:
port(openai, OPENAI_MODEL, pi)

In [74]:
# Use the commands from GPT 5

def compile_and_run():
    subprocess.run(compile_command, check=True, text=True, capture_output=True)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)
    print(subprocess.run(run_command, check=True, text=True, capture_output=True).stdout)

In [75]:
compile_and_run()

Result: 3.141592656089
Execution Time: 0.398667 seconds

Result: 3.141592656089
Execution Time: 0.410239 seconds

Result: 3.141592656089
Execution Time: 0.430209 seconds



In [76]:
port(claude, CLAUDE_MODEL, pi)
compile_and_run()

Result: 3.141592656089
Execution Time: 0.380459 seconds

Result: 3.141592656089
Execution Time: 0.668549 seconds

Result: 3.141592656089
Execution Time: 0.404880 seconds



In [77]:
port(gemini, GEMINI_MODEL, pi)
compile_and_run()

Result: 3.141592656089
Execution Time: 0.399597 seconds

Result: 3.141592656089
Execution Time: 0.625111 seconds

Result: 3.141592656089
Execution Time: 0.390795 seconds

